In [13]:
'''post SQL data clean for eICU-CRD'''
# 手动修改: 

import os, sys
import pandas as pd
pd.set_option('display.max_columns', None)

# 检测运行环境
def in_notebook():
    return 'IPKernelApp' in getattr(globals().get('get_ipython', lambda: None)(), 'config', {})

if in_notebook():
    notebook_dir = os.getcwd()
    src_path = os.path.abspath(os.path.join(notebook_dir, '..'))
else:
    src_path = os.path.abspath(os.path.dirname(os.path.dirname(__file__)))
sys.path.append(src_path) if src_path not in sys.path else None

from src.utils import *
from src.setup import *
from src.data_utils import replace_abnormal_values
eICU_PATH = f'{ROOT}/data/eICU-ICU-sepsis/' # 原始数据


In [14]:
# 获取所有文件路径
filelist = getfiles(eICU_PATH)

# 初始化 DataFrame 列表
df_list = []

# 合并所有表格
for file in filelist:
    fname = os.path.split(file)[1]
    
    # 跳过
    if fname=='eICU既往史数据2(忽略).csv':
        continue
    print(f'正在处理: {fname}')
    df_temp = pd.read_csv(file, index_col='patientunitstayid')
    df_list.append(df_temp)

df = pd.concat(df_list, axis=1)
df.index.name = 'ID'

df.to_excel(f'{DATA}/eICU_merged.xlsx')
df

正在处理: EICU人口统计学数据.csv
正在处理: EICU机械通气数据(有创+无创).csv
正在处理: EICU生命体征数据.csv
正在处理: eICU血气分析指标更新版.csv
正在处理: EICU评分数据.csv
正在处理: eICU诊断数据1.csv
正在处理: EICU预后数据.csv
正在处理: 凝血指标.csv
正在处理: 单纯有创数据.csv
正在处理: 肝肾功离子.csv
正在处理: 脓毒症休克数据.csv
正在处理: 血常规+CRP.csv


,age,gender,height,race,weight,ventilation,ventilation_first_time,heartrate,ibp_diastolic,ibp_mean,ibp_systolic,nibp_diastolic,nibp_mean,nibp_systolic,respiratoryrate,spo2,temperature,first_labaniongap,first_labaniongap_uom,first_labbicarbonate,first_labbicarbonate_uom,first_labfio2,first_labfio2_uom,first_labhco3,first_labhco3_uom,first_lablactate,first_lablactate_uom,first_labo2sat,first_labo2sat_uom,first_labpco2,first_labpco2_uom,first_labph,first_labph_uom,first_labpo2,first_labpo2_uom,apache_score,aps_score,delirium_score,gcs,gcs_eyes,gcs_motor,gcs_verbal,pain_score,bloodcancer,cancer,hivaids,jifacancer,liveryinghua,manxingxinshuai,hospdischargestatus,hosplosday,unitadmitsource,unitdischargelocation,unitdischargestatus,unitlosday,unitstaytype,unittype,first_labfibrinogen,first_labfibrinogen_uom,first_labptinr,first_labptinr_uom,first_labptt,first_labptt_uom,first_labpttratio,first_labpttratio_uom,ventilation,ventilation_first_time,first_glu,first_glu_uom,first_labalt,first_labalt_uom,first_labast,first_labast_uom,first_labbun,first_labbun_uom,first_labchloride,first_labchloride_uom,first_labcreatinine,first_labcreatinine_uom,first_labpotassium,first_labpotassium_uom,first_labsodium,first_labsodium_uom,first_labtotalbilirubin,first_labtotalbilirubin_uom,xiuke,first_labcrp,first_labcrp_uom,first_labhct,first_labhct_uom,first_labhgb,first_labhgb_uom,first_lablymphocytes,first_lablymphocytes_uom,first_labplateletcount,first_labplateletcount_uom,first_labpolys,first_labpolys_uom,first_labrbc,first_labrbc_uom,first_labwbc,first_labwbc_uom
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
141194,68,Male,180.3,Caucasian,73.9,NaN,NaN,108.0,34.0,48.0,84.0,63.0,85.0,115.0,26.0,100.0,38.1,15.0,NaN,19.0,mmol/L,28.0,%,13.0,mmol/L,1.3,mmol/L,NaN,NaN,26.0,mm Hg,7.31,NaN,100.0,mm Hg,70.0,57.0,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alive,9.22,Floor,Floor,Alive,3.34,admit,CTICU,NaN,NaN,NaN,NaN,24.0,sec,NaN,NaN,NaN,NaN,165.0,mg/dL,20.0,Units/L,24.0,Units/L,31.0,mg/dL,105.0,mmol/L,2.51,mg/dL,4.0,mmol/L,135.0,mmol/L,0.4,mg/dL,NaN,NaN,NaN,27.4,%,8.9,g/dL,1.0,%,233.0,K/mcL,98.0,%,3.43,M/mcL,14.1,K/mcL
141197,71,Male,162.6,Caucasian,102.1,NaN,NaN,103.0,NaN,NaN,NaN,68.0,NaN,123.0,24.0,94.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alive,3.86,Emergency Department,Step-Down Unit (SDU),Alive,0.05,admit,Med-Surg ICU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141227,82,Male,185.4,Caucasian,82.2,1.0,1567.0,112.0,58.0,92.0,158.0,66.0,90.0,131.0,48.0,96.0,40.1,20.0,NaN,16.0,mmol/L,21.0,%,16.0,mmol/L,1.4,mmol/L,NaN,NaN,23.0,mm Hg,7.46,NaN,65.0,mm Hg,100.0,83.0,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alive,1.89,Floor,Other Hospital,Alive,1.15,admit,Med-Surg ICU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.0,mg/dL,NaN,NaN,NaN,NaN,30.0,mg/dL,101.0,mmol/L,1.60,mg/dL,4.0,mmol/L,133.0,mmol/L,NaN,NaN,NaN,NaN,NaN,26.2,%,8.2,g/dL,7.0,%,198.0,K/mcL,39.0,%,3.06,M/mcL,42.7,K/mcL
141266,73,Male,188.0,Caucasian,120.4,NaN,NaN,94.0,NaN,78.0,NaN,63.0,NaN,101.0,22.0,97.0,37.4,9.0,NaN,37.0,mmol/L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.0,28.0,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alive,7.75,Emergency Department,Floor,Alive,1.04,admit,Med-Surg ICU,NaN,NaN,NaN,NaN,26.0,sec,NaN,NaN,NaN,NaN,125.0,mg/dL,32.0,Units/L,76.0,Units/L,31.0,mg/dL,100.0,mmol/L,1.60,mg/dL,3.7,mmol/L,142.0,mmol/L,0.8,mg/dL,NaN,NaN,NaN,27.5,%,8.0,g/dL,9.0,%,162.0,K/mcL,82.0,%,3.50,M/mcL,10.0,K/mcL
141288,61,Female,162.6,Caucasian,NaN,NaN,NaN,101.0,53.0,72.0,117.0,54.0,NaN,96.0,45.0,99.0,38.2,13.0,NaN,26.0,mmol/L,100.0,%,24.0,mmol/L,0.9,mmol/L,NaN,NaN,43.0,mm Hg,7.37,NaN,81.0,mm Hg,33.0,22.0,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alive,6.16,Emergency Department,Floor,Alive

# 选择变量

In [15]:
SELECTED_VAR = [
                'age',
                'gender',
                'height',
                'weight',
                # 'BMI', # to derive
                'temperature', # 体温
                'heartrate', # 心率
                'respiratoryrate', # 呼吸频率
                'nibp_systolic', # SBP
                'nibp_diastolic', # DBP
                'nibp_mean', # MAP non-invasive 
                'ibp_mean', # MAP invasive #! (fillna)
                'spo2', # 血氧饱和度

                'cancer',
                'jifacancer',
                'bloodcancer',
                'hivaids',
                'liveryinghua',
                'manxingxinshuai',
                
                'gcs',
                'apache_score', # APACHE IV
                'aps_score', # APS III

                'first_labrbc',  # RBC
                'first_labwbc',  # WBC
                'first_labhgb',  # Hb
                'first_labpolys',  # NE%
                'first_lablymphocytes',  # LYN%
                'first_labplateletcount',  # PLT
                'first_labhct',  # HCT

                'first_labalt', 
                'first_labast', 
                'first_labtotalbilirubin', # STB #! mg/dL
                'first_labbun', # BUN, #! mg/dL
                'first_labcreatinine', # Scr, #! mg/dL
                'first_glu', # Glu, #! mg/dL
                'first_labpotassium', # K
                'first_labsodium', # Na
                'first_labchloride', #? Cl

                # 'first_labcrp', # CRP 缺失过多

                'first_labfibrinogen', # Fg
                'first_labptt', # APTT
                'first_labptinr', # PT_INR

                'first_labph', # PH值
                'first_labpo2', # PaO2
                'first_labfio2', # FiO2
                'first_labbicarbonate', # HCO3-
                'first_labhco3', # HCO3- #! (fillna)
                'first_labpco2', # PaCO2
                'first_lablactate', # Lac 血乳酸(mmol/L)

                'hospdischargestatus', 'unitdischargestatus', 
                'hosplosday', 'unitlosday',
                ]
df_selected = df[SELECTED_VAR].copy()
df_selected.to_excel(f'{DATA}/eICU_selected.xlsx')

In [16]:
df_selected

,age,gender,height,weight,temperature,heartrate,respiratoryrate,nibp_systolic,nibp_diastolic,nibp_mean,ibp_mean,spo2,cancer,jifacancer,bloodcancer,hivaids,liveryinghua,manxingxinshuai,gcs,apache_score,aps_score,first_labrbc,first_labwbc,first_labhgb,first_labpolys,first_lablymphocytes,first_labplateletcount,first_labhct,first_labalt,first_labast,first_labtotalbilirubin,first_labbun,first_labcreatinine,first_glu,first_labpotassium,first_labsodium,first_labchloride,first_labfibrinogen,first_labptt,first_labptinr,first_labph,first_labpo2,first_labfio2,first_labbicarbonate,first_labhco3,first_labpco2,first_lablactate,hospdischargestatus,unitdischargestatus,hosplosday,unitlosday
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
141194,68,Male,180.3,73.9,38.1,108.0,26.0,115.0,63.0,85.0,48.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,15.0,70.0,57.0,3.43,14.1,8.9,98.0,1.0,233.0,27.4,20.0,24.0,0.4,31.0,2.51,165.0,4.0,135.0,105.0,NaN,24.0,NaN,7.31,100.0,28.0,19.0,13.0,26.0,1.3,Alive,Alive,9.22,3.34
141197,71,Male,162.6,102.1,NaN,103.0,24.0,123.0,68.0,NaN,NaN,94.0,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alive,Alive,3.86,0.05
141227,82,Male,185.4,82.2,40.1,112.0,48.0,131.0,66.0,90.0,92.0,96.0,NaN,NaN,NaN,NaN,NaN,NaN,11.0,100.0,83.0,3.06,42.7,8.2,39.0,7.0,198.0,26.2,NaN,NaN,NaN,30.0,1.60,145.0,4.0,133.0,101.0,NaN,NaN,NaN,7.46,65.0,21.0,16.0,16.0,23.0,1.4,Alive,Alive,1.89,1.15
141266,73,Male,188.0,120.4,37.4,94.0,22.0,101.0,63.0,NaN,78.0,97.0,NaN,NaN,NaN,NaN,NaN,NaN,15.0,44.0,28.0,3.50,10.0,8.0,82.0,9.0,162.0,27.5,32.0,76.0,0.8,31.0,1.60,125.0,3.7,142.0,100.0,NaN,26.0,NaN,NaN,NaN,NaN,37.0,NaN,NaN,NaN,Alive,Alive,7.75,1.04
141288,61,Female,162.6,NaN,38.2,101.0,45.0,96.0,54.0,NaN,72.0,99.0,NaN,NaN,NaN,NaN,NaN,NaN,15.0,33.0,22.0,3.62,10.5,11.8,86.0,8.0,175.0,35.4,NaN,NaN,NaN,13.0,0.73,88.0,3.6,140.0,105.0,NaN,NaN,1.10,7.37,81.0,100.0,26.0,24.0,43.0,0.9,Alive,Alive,6.16,1.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3352940,23,Female,165.1,55.3,39.2,126.0,18.0,96.0,52.0,70.0,NaN,86.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,71.0,71.0,4.40,18.9,13.2,NaN,11.0,138.0,39.0,31.0,47.0,0.4,8.0,0.45,73.0,4.5,124.0,93.0,NaN,NaN,NaN,7.44,68.0,98.0,23.0,23.5,35.4,4.0,Alive,Alive,38.22,11.73
3352961,70,Male,177.8,91.0,38.1,71.0,28.0,118.0,56.0,82.0,NaN,94.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,75.0,59.0,4.16,15.4,11.2,NaN,10.0,258.0,34.0,6.0,31.0,0.7,20.0,1.27,90.0,4.2,138.0,106.0,NaN,NaN,NaN,7.52,77.0,32.0,23.0,21.8,26.9,NaN,Alive,Alive,10.93,1.02
3353144,51,Male,180.3,39.2,33.8,58.0,20.0,107.0,76.0,88.0,70.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.0,78.0,3.59,18.1,11.4,NaN,6.0,127.0,34.0,11.0,24.0,0.7,11.0,1.04,155.0,3.9,134.0,106.0,220.0,38.0,1.38,7.44,389.0,100.0,21.0,24.5,34.7,2.3,Alive,Alive,21.96,11.05


In [17]:
df_clean = df_selected[['age']].copy()
df_clean['sex'] = df_selected['gender'].replace({'Male':1, 'Female':0})

df_clean['weight'] = replace_abnormal_values(df_selected['weight'], lower_bound=20, upper_bound=500)  # 体重 异常值排除
df_clean['height'] = df_selected['height'] # 留作缺失填补用， 建模时只留BMI

df_clean['BMI'] = df_clean['weight'] / ((df_selected['height']/100)**2)
df_clean['BMI'] = replace_abnormal_values(df_clean['BMI'], lower_bound=10, upper_bound=70)  # BMI 异常值排除

df_clean['temperature'] = replace_abnormal_values(df_selected['temperature'], lower_bound=32, upper_bound=42)  # 体温 异常值排除
df_clean['heart_rate'] = replace_abnormal_values(df_selected['heartrate'], lower_bound=20, upper_bound=400)  # 心率 异常值排除
df_clean['respir_rate'] = replace_abnormal_values(df_selected['respiratoryrate'], lower_bound=6, upper_bound=60) # 呼吸
df_clean['SBP'] = replace_abnormal_values(df_selected['nibp_systolic'], lower_bound=30, upper_bound=300) # SBP 异常值排除
df_clean['DBP'] = replace_abnormal_values(df_selected['nibp_diastolic'], lower_bound=20, upper_bound=200) # DBP
df_clean['MAP'] = replace_abnormal_values(df_selected['nibp_mean'], lower_bound=20, upper_bound=200).fillna(
                  replace_abnormal_values(df_selected['ibp_mean'], lower_bound=20, upper_bound=200)
                  ) # MAP
df_clean['SPO2'] = replace_abnormal_values(df_selected['spo2'], lower_bound=50, upper_bound=100) # SPO2

df_clean['cancer'] = df_selected['cancer'].fillna(0) 
df_clean['metastatic_cancer'] = df_selected['jifacancer'].fillna(0) 
df_clean['hematologic_cancer'] = df_selected['bloodcancer'].fillna(0) 
df_clean['AIDS'] = df_selected['hivaids'].fillna(0) 
df_clean['liver_cirrhosis'] = df_selected['liveryinghua'].fillna(0)
df_clean['chronic_heart_failure'] = df_selected['manxingxinshuai'].fillna(0)

df_clean['GCS'] = replace_abnormal_values(df_selected['gcs'], lower_bound=0, upper_bound=15) # Glasgow Coma Scale 昏迷评分
df_clean['APS_III'] = replace_abnormal_values(df_selected['aps_score'], lower_bound=0, upper_bound=1000) 
df_clean['APACHE_IV'] = replace_abnormal_values(df_selected['apache_score'], lower_bound=0, upper_bound=np.inf) 

df_clean['RBC'] = replace_abnormal_values(df_selected['first_labrbc'], lower_bound=1, upper_bound=10)
df_clean['WBC'] = replace_abnormal_values(df_selected['first_labwbc'], lower_bound=1, upper_bound=100)
df_clean['Hb'] = replace_abnormal_values(df_selected['first_labhgb']*10, lower_bound=10, upper_bound=250)
df_clean['NE%'] = replace_abnormal_values(df_selected['first_labpolys'], lower_bound=0, upper_bound=100)
df_clean['LYM%'] = replace_abnormal_values(df_selected['first_lablymphocytes'], lower_bound=0, upper_bound=100)
df_clean['PLT'] = replace_abnormal_values(df_selected['first_labplateletcount'], lower_bound=10, upper_bound=1000)
df_clean['HCT'] = replace_abnormal_values(df_selected['first_labhct'], lower_bound=10, upper_bound=80)

df_clean['ALT'] = replace_abnormal_values(df_selected['first_labalt'], lower_bound=0, upper_bound=5000)
df_clean['AST'] = replace_abnormal_values(df_selected['first_labast'], lower_bound=0, upper_bound=5000)
df_clean['STB'] = replace_abnormal_values(df_selected['first_labtotalbilirubin'] * 17.104, lower_bound=0, upper_bound=400) # mg/dL -> umol/L
df_clean['BUN'] = replace_abnormal_values(df_selected['first_labbun'] * 0.357, lower_bound=0, upper_bound=50) # mg/dL -> mmol/L
df_clean['Scr'] = replace_abnormal_values(df_selected['first_labcreatinine'] * 88.4, lower_bound=10, upper_bound=1000) # mg/dL -> umol/L
df_clean['Glu'] = replace_abnormal_values(df_selected['first_glu'] * 0.0555, lower_bound=1, upper_bound=50) # mg/dL -> mmol/L
df_clean['K+'] = replace_abnormal_values(df_selected['first_labpotassium'], lower_bound=1, upper_bound=8)
df_clean['Na+'] = replace_abnormal_values(df_selected['first_labsodium'], lower_bound=120, upper_bound=160)
df_clean['Cl-'] = replace_abnormal_values(df_selected['first_labchloride'], lower_bound=80, upper_bound=120)

df_clean['Fg'] = replace_abnormal_values(df_selected['first_labfibrinogen'] * 0.01, lower_bound=0, upper_bound=10) # mg/dL -> g/L
df_clean['APTT'] = replace_abnormal_values(df_selected['first_labptt'], lower_bound=0, upper_bound=100)
df_clean['PT_INR'] = replace_abnormal_values(df_selected['first_labptinr'], lower_bound=0, upper_bound=20)
df_clean['PH'] = replace_abnormal_values(df_selected['first_labph'], lower_bound=6.5, upper_bound=8.0)
df_clean['PaO2'] = replace_abnormal_values(df_selected['first_labpo2'], lower_bound=30, upper_bound=1000)
df_clean['FiO2'] = replace_abnormal_values(df_selected['first_labfio2'], lower_bound=21, upper_bound=100)
df_clean['PaO2/FiO2'] = replace_abnormal_values(df_clean['PaO2']/df_clean['FiO2'], lower_bound=0, upper_bound=1000)
df_clean['PaCO2'] = replace_abnormal_values(df_selected['first_labpco2'], lower_bound=10, upper_bound=120)
df_clean['HCO3-'] = replace_abnormal_values(df_selected['first_labbicarbonate'], lower_bound=5, upper_bound=50).fillna(
                    replace_abnormal_values(df_selected['first_labhco3'], lower_bound=5, upper_bound=50)
                    ) # HCO3-
df_clean['Lac'] = replace_abnormal_values(df_selected['first_lablactate'], lower_bound=0.1, upper_bound=20)

df_selected['hospital_mortality'] = df_selected['hospdischargestatus'].replace({'Expired':1, 'Alive':0})
df_selected['icu_mortality'] = df_selected['unitdischargestatus'].replace({'Expired':1, 'Alive':0})
df_clean['in_hospital_mortality'] = df_selected[['hospital_mortality', 'icu_mortality']].max(axis=1)
# 排除结局缺失患者
mask_missing_outcome = df_clean['in_hospital_mortality'].isna()
df_clean = df_clean[~mask_missing_outcome].copy()
print(f"排除: {mask_missing_outcome.sum()} 院内死亡结局缺失")

# 排除24h内死亡患者
mask_death_in_24h = (df_selected['hosplosday'] < 1) | (df_selected['unitlosday'] < 1)
df_clean = df_clean[~mask_death_in_24h].copy()
print(f"排除: {mask_death_in_24h.sum()} 入院24小时内死亡")

# 排除缺失率过高的患者
mask_missing_patient = (df_clean.isna().sum(axis=1) / df_clean.shape[1]) >= 0.7
df_clean = df_clean[~mask_missing_patient].copy()
print(f"排除: {mask_missing_patient.sum()} 缺失特征>=70% ")

# 排除缺失率过高的特征
# mask_missing_feature = (df_clean.isna().sum() / df_clean.shape[0]) >= 0.7
# exclude_feature = df_clean.columns[mask_missing_feature]
# print(f"排除: {list(exclude_feature)} 数据缺失率>=70% ")
# df_clean = df_clean.drop(columns=exclude_feature)
for feature, missing_rate in (df_clean.isna().sum() / df_clean.shape[0]).to_dict().items():
    if missing_rate >= 0.9:
        mark = '***'
    elif missing_rate >= 0.7:
        mark = '**'
    elif missing_rate >= 0.5:
        mark = '*'
    else:
        mark = ''
    print(f'{feature:>20} | {missing_rate*100:.2f}%  {mark}')

mask_in_hos_death = (df_clean['in_hospital_mortality']==1)
print(f'住院死亡: {mask_in_hos_death.sum()} ({mask_in_hos_death.sum()/len(df_clean)*100:.2f}%)')
df_clean.to_csv(f'{DATA}/eICU_clean.tsv.gz', sep='\t', compression='gzip')

              weight | 11 (0.04%)	 abnormal couunts 
                 BMI | 180 (0.72%)	 abnormal couunts 
         temperature | 56 (0.22%)	 abnormal couunts 
           heartrate | 0 (0.00%)	 abnormal couunts 
     respiratoryrate | 56 (0.22%)	 abnormal couunts 
       nibp_systolic | 1 (0.00%)	 abnormal couunts 
      nibp_diastolic | 31 (0.12%)	 abnormal couunts 
           nibp_mean | 11 (0.04%)	 abnormal couunts 
            ibp_mean | 93 (0.37%)	 abnormal couunts 
                spo2 | 32 (0.13%)	 abnormal couunts 
                 gcs | 0 (0.00%)	 abnormal couunts 
           aps_score | 121 (0.48%)	 abnormal couunts 
        apache_score | 121 (0.48%)	 abnormal couunts 
        first_labrbc | 6 (0.02%)	 abnormal couunts 
        first_labwbc | 413 (1.65%)	 abnormal couunts 
        first_labhgb | 0 (0.00%)	 abnormal couunts 
      first_labpolys | 0 (0.00%)	 abnormal couunts 
first_lablymphocytes | 0 (0.00%)	 abnormal couunts 
first_labplateletcount | 100 (0.40%)	 abnormal co

In [18]:
df_clean

,age,sex,weight,height,BMI,temperature,heart_rate,respir_rate,SBP,DBP,MAP,SPO2,cancer,metastatic_cancer,hematologic_cancer,AIDS,liver_cirrhosis,chronic_heart_failure,GCS,APS_III,APACHE_IV,RBC,WBC,Hb,NE%,LYM%,PLT,HCT,ALT,AST,STB,BUN,Scr,Glu,K+,Na+,Cl-,Fg,APTT,PT_INR,PH,PaO2,FiO2,PaO2/FiO2,PaCO2,HCO3-,Lac,in_hospital_mortality
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
141194,68,1,73.9,180.3,22.732803,38.1,108.0,26.0,115.0,63.0,85.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,57.0,70.0,3.43,14.1,89.0,98.0,1.0,233.0,27.4,20.0,24.0,6.8416,11.067,221.884,9.1575,4.0,135.0,105.0,NaN,24.0,NaN,7.31,100.0,28.0,3.571429,26.0,19.0,1.3,0.0
141227,82,1,82.2,185.4,23.914007,40.1,112.0,48.0,131.0,66.0,90.0,96.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,83.0,100.0,3.06,42.7,82.0,39.0,7.0,198.0,26.2,NaN,NaN,NaN,10.710,141.440,8.0475,4.0,133.0,101.0,NaN,NaN,NaN,7.46,65.0,21.0,3.095238,23.0,16.0,1.4,0.0
141266,73,1,120.4,188.0,34.065188,37.4,94.0,22.0,101.0,63.0,78.0,97.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,28.0,44.0,3.50,10.0,80.0,82.0,9.0,162.0,27.5,32.0,76.0,13.6832,11.067,141.440,6.9375,3.7,142.0,100.0,NaN,26.0,NaN,NaN,NaN,NaN,NaN,NaN,37.0,NaN,0.0
141288,61,0,NaN,162.6,NaN,38.2,101.0,45.0,96.0,54.0,72.0,99.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,22.0,33.0,3.62,10.5,118.0,86.0,8.0,175.0,35.4,NaN,NaN,NaN,4.641,64.532,4.8840,3.6,140.0,105.0,NaN,NaN,1.10,7.37,81.0,100.0,0.810000,43.0,26.0,0.9,0.0
141304,70,1,NaN,165.0,NaN,38.5,118.0,28.0,106.0,55.0,82.0,95.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,41.0,67.0,2.79,12.8,78.0,91.0,3.0,260.0,25.5,NaN,NaN,NaN,17.136,181.220,7.1595,5.8,140.0,105.0,NaN,NaN,NaN,7.38,89.0,NaN,NaN,43.0,29.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3352940,23,0,55.3,165.1,20.287615,39.2,126.0,18.0,96.0,52.0,70.0,86.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,71.0,71.0,4.40,18.9,132.0,NaN,11.0,138.0,39.0,31.0,47.0,6.8416,2.856,39.780,4.0515,4.5,124.0,93.0,NaN,NaN,NaN,7.44,68.0,98.0,0.693878,35.4,23.0,4.0,0.0
3352961,70,1,91.0,177.8,28.785772,38.1,71.0,28.0,118.0,56.0,82.0,94.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,59.0,75.0,4.16,15.4,112.0,NaN,10.0,258.0,34.0,6.0,31.0,11.9728,7.140,112.268,4.9950,4.2,138.0,106.0,NaN,NaN,NaN,7.52,77.0,32.0,2.406250,26.9,23.0,NaN,0.0
3353144,51,1,39.2,180.3,12.058537,33.8,58.0,20.0,107.0,76.0,88.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,78.0,83.0,3.59,18.1,114.0,NaN,6.0,127.0,34.0,11.0,24.0,11.9728,3.927,91.936,8.6025,3.9,134.0,106.0,2.20,38.0,1.38,7.44,389.0,100.0,3.890000,34.7,21.0,2.3,0.0
